In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import sys
import os
import torch
import argparse
import data
import util
import torch.nn as nn
import torch.optim as optim

from models import nin_store as nin
from torch.autograd import Variable
import tqdm
import time
import numpy as np

def vectorRelative(origin, failure):
    return ((origin - failure).pow(2).sum())/(origin.pow(2).sum())

def totalRelative(origin, failure):
    return ((origin - failure)/origin).pow(2).sum()/(len(origin.view(-1)))

def compare_activations(origin, failure):
    batches = len(origin)
    layers = len(origin[0])
    difference = torch.zeros(9)
    for i in range(batches):
        for j in range(layers): 
            ori = origin[i][j].to(device)
            fal = failure[i][j].to(device)
            difference[j] += totalRelative(ori,fal)
            del(ori)
            del(fal)
    
    return difference

def load_pretrained(filePath):
    model = nin.Net()
    pretrained_model = torch.load(filePath)
    useState_dict = model.state_dict()
    preState_dict = pretrained_model['state_dict']
    useKeys = useState_dict.keys()
    preKeys = preState_dict.keys()
    j = 0
    for key in useKeys:
        if key.find('num_batches_tracked') == -1:
            useState_dict[key].data = preState_dict[preKeys[j]].data
            j +=1
    model.load_state_dict(useState_dict)
    model.to(device)
    return model

def BitInverse(i, key, shape):
    
    model = load_pretrained('nin.best.pth.tar')
    bin_op = util.BinOp(model)
    model.eval()
    bin_op.binarization()
    state_dict = model.state_dict()
    

    if len(shape) == 4:
        size1 = shape[1]
        size2 = shape[2]
        size3 = shape[3]
        (state_dict[key][int(i/size1/size2/size3)][int(i/size2/size3%size1)][int(i/size3%size2)][int(i%size3)]).mul_(-1)

    if len(shape) == 1:
        state_dict[key][i].mul_(-1)

    if len(shape) == 2:
        size = state_dict[key].shape[1]
        (state_dict[key][int(i/size)][i%size]).mul_(-1)
    
    model.load_state_dict(state_dict)
    return model

In [2]:
def test(i, key, shape, memoryData = None):
    activations = []
    test_loss = 0
    correct = 0
    
    model = BitInverse(i, key, shape)
    model.eval()
    with torch.no_grad():
        for data, target in memoryData:
            data, target = data.to(device), target.to(device)

            output, activation = model(data)
            #test_loss += criterion(output, target).data.item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()
            activations += [activation]
            
    bin_op.restore()
    acc = 100. * float(correct) / float(len(testloader.dataset))

    return acc, activations

In [3]:
# test environment

device = torch.device('cuda:1' if torch.cuda.is_available() else "cpu")

# set the seed
torch.manual_seed(1)
torch.cuda.manual_seed(1)

# prepare the data
if not os.path.isfile('./data/'+'/train_data'):
    # check the data path
    raise Exception\
            ('Please assign the correct data path with --data <DATA_PATH>')

testset = data.dataset(root='./data/', train=False)
indices = np.load("subset_CIFAR10.npy")
testloader = torch.utils.data.DataLoader(testset,
                             batch_size=512, shuffle=False, num_workers=4)

# define classes
classes = ('plane', 'car', 'bird', 'cat',
        'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# define the model
model = load_pretrained('nin.best.pth.tar')

# define the binarization operator
bin_op = util.BinOp(model)

# do the evaluation if specified
bestAcc = 86.28
memoryData = []

find_key = "conv1.weight"
print(find_key)
state_dict = model.state_dict()

for key in state_dict.keys():
    if key.find(find_key) != -1:
        total = 1
        shape = state_dict[key].shape
        use_key = key
        for t in range(len(state_dict[key].shape)):
            total *= state_dict[key].shape[t]     

for data_batches in tqdm.tqdm_notebook(testloader, leave = False):
    memoryData += [data_batches]

conv1.weight


In [4]:
# tests
#indices = [13668, 12162, 7373, 5867, 8631, 12611, 6580, 1312, 12933, 1287]
activationFolder = './Activations/'
resultFolder = './Results/neg_div_4/'
upperBound = 0.7
lowerBound = 0.2
lowerCut = 0.1
origin = torch.load(activationFolder + 'Correct_Activations.pt')
losses = np.load(resultFolder + '0.weight.neg.npy')

In [13]:
lowerBound = 0.15
lowerCut = 0.1
upperIndices = []
lowerIndices = []
j = 0
for i in range(len(losses[:,1])):
    if losses[i,1] > upperBound:
        upperIndices += [losses[i,0]]
    elif losses[i,1] < lowerBound and losses[i,1] > lowerCut:
        lowerIndices += [losses[i,0]]
    elif losses[i,1] <= lowerCut:
        if j > 100:
            j += 1
            lowerIndices += [losses[i,0]]
            

In [15]:
with tqdm.tqdm_notebook(upperIndices) as Loader:
    differences = []
    for i in Loader:
        acc, activation = test(i, use_key, shape = shape, memoryData = memoryData)
        loss = bestAcc - acc
        Loader.set_description("L: %.2f"%(loss))
        differences += [[compare_activations(origin, activation), loss, i]]
        torch.save(differences, 'upperDifferencestotalRelative.pt')

In [17]:
with tqdm.tqdm_notebook(lowerIndices) as Loader:
    differences = []
    j = 0
    for i in Loader:
        acc, activation = test(i, use_key, shape = shape, memoryData = memoryData)
        loss = bestAcc - acc
        Loader.set_description("L: %.2f"%(loss))
        differences += [[compare_activations(origin, activation), loss, i]]
        torch.save(differences, 'lowerDifferencestotalRelative.pt')

In [8]:
fuccc = torch.zeros([1,2,3])

In [10]:
print(len(fuccc.view(-1)))

6


In [67]:
def getDiff(diff):
    output = []
    if len(diff) == 165 or len(diff) == 2259:
        for i in range(0,len(diff),3):
            output += [diff[i]]
    else:
        for i in range(0,len(diff)):
            output += [diff[i][0]]
    return output

def calcAvg(Diff):
    Total = torch.zeros(len(Diff[0]))
    for item in Diff:
        Total += item
    avg = Total/len(Diff)
    print (avg)

def fileAvg(filename):
    Data = torch.load(filename)
    Differences = getDiff(Data)
    calcAvg(Differences)

In [68]:
fileAvg('Activations/upperDifferences.pt')
fileAvg('Activations/upperDifferencesVectorRelative.pt')
fileAvg('Activations/lowerDifferences.pt')
fileAvg('Activations/lowerDifferencesVectorRelative.pt')

tensor([4.8501e-03, 9.0787e-01, 7.8004e+00, 1.2709e+02, 2.9573e+01, 1.6705e+02,
        1.0421e+03, 4.2285e+01, 8.7858e+02])
tensor([0.0091, 0.0059, 0.0760, 0.0888, 0.2937, 1.2942, 0.6536, 1.7197, 1.6381])
tensor([1.1118e-03, 3.5555e-01, 2.9627e+00, 4.7913e+01, 1.4386e+01, 8.7962e+01,
        5.9837e+02, 2.7590e+01, 5.6897e+02])
tensor([0.0021, 0.0023, 0.0289, 0.0335, 0.1429, 0.6815, 0.3753, 1.1221, 1.0608])


In [66]:
Data = torch.load('Activations/lowerDifferences.pt')
print (len(Data))

2259


In [6]:
for i in range(0,2,4):
    print (i)

0
